In [1]:
import scanpy as sc
import pandas as pd

In [2]:
%load_ext memory_profiler

# Now use it to monitor memory of a single line:

adata_path = '../../processed_data2/BoneMarrow_May2024_outliers_removed_Palantir.h5ad'

%memit adata = sc.read_h5ad(adata_path)

peak memory: 35725.25 MiB, increment: 35477.44 MiB


In [6]:
adata

AnnData object with n_obs × n_vars = 75386 × 26708
    obs: 'n_genes_by_counts_prefilter', 'log1p_n_genes_by_counts_prefilter', 'total_counts_prefilter', 'log1p_total_counts_prefilter', 'pct_counts_in_top_50_genes_prefilter', 'pct_counts_in_top_100_genes_prefilter', 'pct_counts_in_top_200_genes_prefilter', 'pct_counts_in_top_500_genes_prefilter', 'Cluster_Fine', 'Cluster_Coarse', 'Annotation', 'Sample', 'Barcode', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'palantir_pseudotime', 'palantir_entropy'
    var: 'n_cells_by_counts_prefilter', 'mean_counts_prefilter', 'log1p_mean_counts_prefilter', 'pct_dropout_by_counts_prefilter', 'total_counts_prefilter', 'log1p_total_counts_prefilter', 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_var

In [12]:
gene_list = ['CD3E', 'CD4', 'CD46']

sc.tl.score_genes(adata, gene_list)

scores1 = adata.obs['score']

In [13]:
del adata

In [16]:
import numpy as np
import anndata as ad

def score_genes_light(adata, gene_list, score_name="gene_score", inplace=True):
    """
    Compute average expression of a gene list in backed mode without loading full .X.

    Parameters:
    - adata: AnnData object (can be backed)
    - gene_list: list of gene names (must match adata.var_names)
    - score_name: column name to store in adata.obs
    - inplace: if True, stores in adata.obs; else returns the array

    Returns:
    - If inplace=False: returns numpy array of gene signature scores
    """
    if adata.is_view:
        adata = adata.copy()

    # Get indices of available genes
    var_names = list(adata.var_names)
    gene_indices = [var_names.index(g) for g in gene_list if g in var_names]
    if len(gene_indices) == 0:
        raise ValueError("None of the genes were found in adata.var_names.")

    print(f"Scoring {len(gene_indices)} genes out of {len(gene_list)} provided...")

    n_cells = adata.n_obs
    scores = np.zeros(n_cells)

    for idx in gene_indices:
        gene_expr = adata[:, idx].X[:]
        scores += np.ravel(gene_expr)

    scores /= len(gene_indices)

    if inplace:
        try:
            adata.obs[score_name] = scores
        except ValueError:
            raise ValueError("Cannot write to .obs in 'read-only' mode. Reopen with backed='r+'")
    else:
        return scores


In [5]:
%load_ext memory_profiler

import anndata as ad
adata_path = '../../processed_data2/BoneMarrow_May2024_outliers_removed_Palantir.h5ad'

In [6]:
%memit adata = ad.read_h5ad(adata_path, backed="r")

peak memory: 32745.82 MiB, increment: 32606.84 MiB


In [7]:
print(type(adata.X))             # Should be numpy array, not sparse
print(adata.file)                # Will be None if loaded into memory

<class 'anndata._core.sparse_dataset.CSRDataset'>
Backing file manager of file ../../processed_data2/BoneMarrow_May2024_outliers_removed_Palantir.h5ad.


In [10]:
adata_raw = adata.raw.to_adata()

AttributeError: 'CSRDataset' object has no attribute 'copy'